In [45]:
import torch
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [46]:
df = pd.read_csv('./traces.csv')
df.head()

,fno,xmin,ymin,xmax,ymax,confidence,cls
0,0,815.328613,414.294556,1023.535522,710.458862,0.840505,1.0
1,1,815.099548,416.698364,1024.056885,710.406616,0.839022,1.0
2,2,816.624023,416.424591,1025.664185,707.520020,0.802077,1.0
3,3,815.463745,414.316956,1025.545410,707.625366,0.820569,1.0
4,11,818.357239,411.877686,1025.415894,708.695923,0.819219,1.0


In [47]:
df['fno'].count()

438

In [48]:
df_conf_filtered = df[df['confidence'] > 0.7].reset_index(drop=True)
df_conf_filtered['fno'].count()

438

In [49]:
df_conf_filtered.dtypes

fno             int64
xmin          float64
ymin          float64
xmax          float64
ymax          float64
confidence    float64
cls           float64
dtype: object

In [50]:
df_conf_filtered.head()

,fno,xmin,ymin,xmax,ymax,confidence,cls
0,0,815.328613,414.294556,1023.535522,710.458862,0.840505,1.0
1,1,815.099548,416.698364,1024.056885,710.406616,0.839022,1.0
2,2,816.624023,416.424591,1025.664185,707.520020,0.802077,1.0
3,3,815.463745,414.316956,1025.545410,707.625366,0.820569,1.0
4,11,818.357239,411.877686,1025.415894,708.695923,0.819219,1.0


In [51]:
cap = cv2.VideoCapture('../videos/capture.mp4')

In [52]:
def mark_frame(cap, fno, pos, cls):
    cap.set(cv2.CAP_PROP_POS_FRAMES, fno)
    res, frame = cap.read()
    if not res:
        return None

    print(pos[0:2])
    print(pos[2:4])

    img = cv2.rectangle(frame, pos[0:2], pos[2:4], (255, 0, 0), 6)
    img = cv2.putText(img, f'{cls}', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
    return img

def mark_and_show(cap, fno, pos, cls):
    img = mark_frame(cap, fno, pos, cls) 
    rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.imshow(rgb)
    plt.show()


In [53]:
record = df_conf_filtered.iloc[0, :].to_numpy().astype(np.int32)
record

array([   0,  815,  414, 1023,  710,    0,    1], dtype=int32)

In [54]:
marked = mark_frame(cap, record[0], record[1:5], record[5])

[815 414]
[1023  710]


In [55]:
# parameters
window_size = 5
with open('./window_size.txt', 'r') as f:
    s = f.read()
    window_size = int(s)

In [56]:
# prepare x axis
labels = df_conf_filtered.columns.drop(['fno', 'confidence'])
new_labels = []

for i in range(window_size):
    for label in labels:
        new_labels.append(f'{i}_{label}')

new_labels.append('waving')
print(new_labels)
print(len(new_labels))

['0_xmin', '0_ymin', '0_xmax', '0_ymax', '0_cls', '1_xmin', '1_ymin', '1_xmax', '1_ymax', '1_cls', '2_xmin', '2_ymin', '2_xmax', '2_ymax', '2_cls', '3_xmin', '3_ymin', '3_xmax', '3_ymax', '3_cls', '4_xmin', '4_ymin', '4_xmax', '4_ymax', '4_cls', '5_xmin', '5_ymin', '5_xmax', '5_ymax', '5_cls', '6_xmin', '6_ymin', '6_xmax', '6_ymax', '6_cls', '7_xmin', '7_ymin', '7_xmax', '7_ymax', '7_cls', '8_xmin', '8_ymin', '8_xmax', '8_ymax', '8_cls', '9_xmin', '9_ymin', '9_xmax', '9_ymax', '9_cls', '10_xmin', '10_ymin', '10_xmax', '10_ymax', '10_cls', '11_xmin', '11_ymin', '11_xmax', '11_ymax', '11_cls', '12_xmin', '12_ymin', '12_xmax', '12_ymax', '12_cls', '13_xmin', '13_ymin', '13_xmax', '13_ymax', '13_cls', '14_xmin', '14_ymin', '14_xmax', '14_ymax', '14_cls', '15_xmin', '15_ymin', '15_xmax', '15_ymax', '15_cls', '16_xmin', '16_ymin', '16_xmax', '16_ymax', '16_cls', '17_xmin', '17_ymin', '17_xmax', '17_ymax', '17_cls', '18_xmin', '18_ymin', '18_xmax', '18_ymax', '18_cls', '19_xmin', '19_ymin', '

In [57]:
import json

f = open('../data/timeseries/waving.json')
ts_result = json.load(f)
f.close()

In [58]:
results = ts_result[0]['annotations'][0]['result']
results

waving_ranges = []

for result in results:
    waving_ranges.append((result['value']['start'], result['value']['end']))

In [59]:
waving_ranges

[(68, 150), (320, 380), (490, 551)]

In [60]:
length = df_conf_filtered.shape[0]

target = pd.DataFrame([], columns=new_labels)
for i in range(0, length-window_size):
    sub_df = df_conf_filtered.iloc[i:i+window_size, :]    
    if sub_df.iloc[window_size-1]['fno'] - sub_df.iloc[0]['fno'] == window_size-1:
        waving = False
        for rg in waving_ranges:
            # if sub_df.iloc[window_size-1]['fno'] <= rg[1] and sub_df.iloc[0]['fno'] >= rg[0]:
            #     waving = True
            #     break
            if sub_df.iloc[window_size-1]['fno'] <= rg[1] and sub_df.iloc[window_size-1]['fno'] >= rg[0]:
                waving = True
                break
        
        
        sub_df = sub_df.drop(['fno', 'confidence'], axis=1)
        sub_widen = np.concatenate([
            sub_df.to_numpy().reshape(window_size*len(sub_df.columns)),
            np.array([waving])
        ]).reshape(1, window_size*len(sub_df.columns) + 1)
        sub_df_widen = pd.DataFrame(sub_widen, columns=new_labels)

        target = pd.concat([target, sub_df_widen])

In [61]:
target

,0_xmin,0_ymin,0_xmax,0_ymax,0_cls,1_xmin,1_ymin,1_xmax,1_ymax,1_cls,...,18_ymin,18_xmax,18_ymax,18_cls,19_xmin,19_ymin,19_xmax,19_ymax,19_cls,waving
0,447.183441,367.721252,762.262268,609.651672,3.0,452.539215,329.056763,771.376282,595.739868,3.0,...,66.607124,934.796387,407.111694,3.0,664.719910,61.861820,936.672729,402.966095,3.0,0.0
0,452.539215,329.056763,771.376282,595.739868,3.0,469.208527,293.963135,792.578796,565.052612,3.0,...,61.861820,936.672729,402.966095,3.0,665.332092,61.366737,939.263184,403.530151,3.0,0.0
0,469.208527,293.963135,792.578796,565.052612,3.0,484.448669,253.992157,781.193726,541.653442,3.0,...,61.366737,939.263184,403.530151,3.0,667.906372,61.061520,939.203735,403.717316,3.0,0.0
0,484.448669,253.992157,781.193726,541.653442,3.0,488.186737,221.486725,798.628296,527.634033,3.0,...,61.061520,939.203735,403.717316,3.0,672.819275,60.724026,942.629761,400.983856,3.0,0.0
0,488.186737,221.486725,798.628296,527.634033,3.0,503.780029,201.353729,823.501465,515.097473,3.0,...,60.724026,942.629761,400.983856,3.0,670.284912,64.300781,948.820251,402.182190,3.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,688.950684,400.690857,946.681030,704.685608,1.0,688.416260,405.174713,947.843872,703.570312,1.0,...,400.851044,948.700256,704.219666,1.0,702.851440,399.100464,949.783997,700.597107,1.0,0.0
0,688.416260,405.174713,947.843872,703.570312,1.0,685.359314,405.131866,945.665894,704.193481,1.0,...,399.100464,949.783997,700.597107,1.0,701.266479,395.763855,949.154297,702.368164,1.0,0.0
0,685.359314,405.131866,945.665894,704.193481,1.0,687.570740,408.475189,952.427490,704.723755,1.0,...,395.763855,949.154297,702.368164,1.0,700.758301,395.922302,950.325012,699.133972,1.0,0.0
0,687.570740,408.475189,952.427490,704.723755,1.0,691.203064,411.794220,951.377380,707.781860,1.0,...,395.922302,950.325012,699.133972,1.0,699.762573,397.087250,951.128723,701.405823,1.0,0.0


In [62]:
target.to_csv('./traces_seq.csv', index=False)

In [63]:
with open('./window_size.txt', 'w') as f:
    f.write(f'{window_size}')

In [64]:
a = np.array([1, 2, 3])

In [65]:
np.concatenate([a, np.array([1])], axis=0)

array([1, 2, 3, 1])

In [66]:
sub_df = df_conf_filtered.iloc[0:5, :]
sub_df.iloc[4]['fno'] > waving_ranges[0][0]

False